In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/model/Prediction stack overflow/train.csv")
df_valid = pd.read_csv("/content/drive/MyDrive/model/Prediction stack overflow/valid.csv")

In [ ]:
df_train['text'] = df_train['Title']+" "+df_train['Body']
df_valid['text'] = df_valid['Title']+" "+df_valid['Body']

In [ ]:
df_train

,Id,Title,Body,Tags,CreationDate,Y,text
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE,Java: Repeat Task Every Random Seconds <p>I'm ...
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ,Why are Java Optionals immutable? <p>I'd like ...
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ,Text Overlay Image with Darkened Opacity React...
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ,Why ternary operator in swift is so picky? <p>...
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ,hide/show fab with scale animation <p>I'm usin...
...,...,...,...,...,...,...,...
44995,60461435,Convert List<String> to string C# - asp.net - ...,<p>I am new to this and I am asking for help t...,<c#><asp.net><sql-server>,2020-02-29 02:22:18,LQ_CLOSE,Convert List<String> to string C# - asp.net - ...
44996,60461754,Does Python execute code from the top or botto...,<p>I am working on learning Python and was won...,<python>,2020-02-29 03:33:59,LQ_CLOSE,Does Python execute code from the top or botto...
44997,60462001,how to change payment date in Azure?,<p>It looks like it costs 8 days per month in ...,<azure><billing>,2020-02-29 04:34:16,LQ_CLOSE,how to change payment date in Azure? <p>It loo...
44998,60465318,how to implement fill in the blank in Swift,"<p>""I _____ any questions.""</p>\n\n<p>I want t...",<ios><swift>,2020-02-29 12:50:43,LQ_CLOSE,how to implement fill in the blank in Swift <p...


In [ ]:
df_train = df_train[['text', 'Y']]
df_valid = df_valid[['text', 'Y']]

In [ ]:
df_valid

,text,Y
0,How to get all the child records from differen...,LQ_EDIT
1,Retrieve all except some data of the another t...,LQ_EDIT
2,Pandas: read_html <p>I'm trying to extract US ...,HQ
3,"Reader Always gimme NULL I'm so new to C#, I w...",LQ_EDIT
4,php rearrange array elements based on conditio...,LQ_EDIT
...,...,...
14995,How can I align two flex boxes to follow each ...,LQ_CLOSE
14996,C++ The correct way to multiply an integer and...,LQ_CLOSE
14997,WHY DJANGO IS SHOWING ME THIS ERROR WHEN I TRY...,LQ_EDIT
14998,PHP - getting the content of php page <p>I hav...,LQ_CLOSE


In [ ]:
import re

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = text.lower()
    return text

def remove_digits(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_underscore(text):
    return text.replace('_', ' ')

def filter_words(text):
    return ' '.join(word for word in text.split() if len(word) > 1)

In [ ]:
a = df_train['text'][34]

In [ ]:
a

"undefined reference to SIGN in C using GCC <p>Many questions have been asked here about functions from math.h not being found in C code during compilation and I am facing a similar problem. Having gone through all of them, none seems to apply to my problem. I have some C code in a file called test1.c - </p>\n\n<pre><code>#include &lt;math.h&gt;\nint main(void) {\n  int j = SQR(4);\n  int i = (int)(sqrt(16.0));\n}\n</code></pre>\n\n<p>Now, I compile this using - </p>\n\n<pre><code>gcc -c -o ../../obj/test1.o test1.c -I../../include\n</code></pre>\n\n<p>And it produces the .o file. </p>\n\n<p>Now, I try and create the executable using - </p>\n\n<pre><code>gcc -o ../../bin/test1../../obj/test1.o -I../../include -lm\n</code></pre>\n\n<p>But this gives me the error below (most previous questions were solved by the -lm switch but I have that) - </p>\n\n<pre><code>../../obj/test1.o:test1.c:(.text+0x13): undefined reference to `SQR'\n../../obj/test1.o:test1.c:(.text+0x13): relocation truncate

In [ ]:
a = remove_special_chars(a)
a = remove_digits(a)
a = remove_underscore(a)
a = filter_words(a)

In [ ]:
a

'undefined reference to sign in using gcc many questions have been asked here about functions from math not being found in code during compilation and am facing similar problem having gone through all of them none seems to apply to my problem have some code in file called test pre code include lt math gt int main void int sqr int int sqrt code pre now compile this using pre code gcc obj test test include code pre and it produces the file now try and create the executable using pre code gcc bin test obj test include lm code pre but this gives me the error below most previous questions were solved by the lm switch but have that pre code obj test test text undefined reference to sqr obj test test text relocation truncated to fit pc against undefined symbol sqr code pre strangely sqrt works but host of other functions like sqr sign etc lead to problems the case doesn matter note that these commands were generated by make file pre code idir include cc gcc cflags idir odir obj ldir lib libs 

In [ ]:
df_train["text"] = df_train["text"].apply(remove_special_chars)
df_train["text"] = df_train["text"].apply(remove_digits)
df_train["text"] = df_train["text"].apply(remove_underscore)
df_train['text'] = df_train['text'].apply(filter_words)

df_valid["text"] = df_valid["text"].apply(remove_special_chars)
df_valid["text"] = df_valid["text"].apply(remove_digits)
df_valid["text"] = df_valid["text"].apply(remove_underscore)
df_valid['text'] = df_valid['text'].apply(filter_words)

<ipython-input-15-7a07a4d6ad28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["text"] = df_train["text"].apply(remove_special_chars)
<ipython-input-15-7a07a4d6ad28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["text"] = df_train["text"].apply(remove_digits)
<ipython-input-15-7a07a4d6ad28>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [ ]:
df_valid

,text,Y
0,how to get all the child records from differen...,LQ_EDIT
1,retrieve all except some data of the another t...,LQ_EDIT
2,pandas read html trying to extract us states f...,HQ
3,reader always gimme null so new to wanna make ...,LQ_EDIT
4,php rearrange array elements based on conditio...,LQ_EDIT
...,...,...
14995,how can align two flex boxes to follow each ot...,LQ_CLOSE
14996,the correct way to multiply an integer and dou...,LQ_CLOSE
14997,why django is showing me this error when try t...,LQ_EDIT
14998,php getting the content of php page have contr...,LQ_CLOSE


In [ ]:
X_train = df_train['text'].copy()
y_train = df_train['Y'].copy()

In [ ]:
X_valid = df_valid['text'].copy()
y_valid = df_valid['Y'].copy()

In [ ]:
y_train = y_train.map({'LQ_CLOSE': 0, 'LQ_EDIT': 1, 'HQ': 2})
y_valid = y_valid.map({'LQ_CLOSE': 0, 'LQ_EDIT': 1, 'HQ': 2})

In [ ]:
y_valid

0        1
1        1
2        2
3        1
4        1
        ..
14995    0
14996    0
14997    1
14998    0
14999    0
Name: Y, Length: 15000, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=1)

print('X_valid shape:', X_valid.shape)
print('y_valid shape:', y_valid.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_valid shape: (7500,)
y_valid shape: (7500,)
X_test shape: (7500,)
y_test shape: (7500,)


In [ ]:
y_test.value_counts()

0    2511
2    2506
1    2483
Name: Y, dtype: int64

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
X_train_input_ids = np.zeros((len(X_train), 256))
X_train_attn_masks = np.zeros((len(y_train), 256))

X_valid_input_ids = np.zeros((len(X_valid), 256))
X_valid_attn_masks = np.zeros((len(X_valid), 256))

X_test_input_ids = np.zeros((len(X_test), 256))
X_test_attn_masks = np.zeros((len(y_test), 256))

In [ ]:
def generate_training_data(X_train, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_train)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks


def generate_training_data(X_valid, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_valid)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

def generate_training_data(X_test, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(X_test)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_train_input_ids, X_train_attn_masks = generate_training_data(X_train, X_train_input_ids, X_train_attn_masks, tokenizer)
     
X_valid_input_ids, X_valid_attn_masks = generate_training_data(X_valid, X_valid_input_ids, X_valid_attn_masks, tokenizer)

X_test_input_ids, X_test_attn_masks = generate_training_data(X_test, X_test_input_ids, X_test_attn_masks, tokenizer)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
X_train[0]

'java repeat task every random seconds already familiar with repeating tasks every seconds by using java util timer and java util timertask but lets say want to print hello world to the console every random seconds from unfortunately in bit of rush and don have any code to show so far any help would be apriciated'

In [ ]:
X_train_input_ids[0]

array([  101.,  9262.,  9377.,  4708.,  2296.,  6721.,  3823.,  2525.,
        5220.,  2007., 15192.,  8518.,  2296.,  3823.,  2011.,  2478.,
        9262., 21183.,  4014., 25309.,  1998.,  9262., 21183.,  4014.,
       25309., 10230.,  2243.,  2021., 11082.,  2360.,  2215.,  2000.,
        6140.,  7592.,  2088.,  2000.,  1996., 10122.,  2296.,  6721.,
        3823.,  2013.,  6854.,  1999.,  2978.,  1997.,  5481.,  1998.,
        2123.,  2031.,  2151.,  3642.,  2000.,  2265.,  2061.,  2521.,
        2151.,  2393.,  2052.,  2022., 19804., 24108.,  3064.,   102.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [ ]:
labels_train = np.zeros((len(y_train), 3))
print(labels_train.shape)

labels_valid = np.zeros((len(y_valid), 3))
print(labels_valid.shape)

labels_test = np.zeros((len(y_test), 3))
print(labels_test.shape)

(45000, 3)
(7500, 3)
(7500, 3)


In [ ]:
labels_train[np.arange(len(y_train)), y_train.values] = 1

labels_valid[np.arange(len(y_valid)), y_valid.values] = 1

labels_test[np.arange(len(y_test)), y_test.values] = 1

In [ ]:
labels_train

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [ ]:
bert_base = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_output = bert_base(input_ids, attention_mask=attention_mask)

cls_token = bert_output.last_hidden_state[:, 0, :]

hidden_layer_01 = tf.keras.layers.Dense(64, activation='relu')(cls_token)
output_layer = tf.keras.layers.Dense(3, activation='softmax')(hidden_layer_01)
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_model_1 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 256, 768),                                                   
                                 hidden_states=None                                         

In [ ]:
history = model.fit(
    x=[X_train_input_ids, X_train_attn_masks],
    y=labels_train,
    batch_size=16,
    epochs=2,
    validation_data=([X_valid_input_ids, X_valid_attn_masks], labels_valid)
)


Epoch 1/2
2813/2813 [==============================] - 1289s 450ms/step - loss: 0.4159 - accuracy: 0.8200 - val_loss: 0.3403 - val_accuracy: 0.8567
Epoch 2/2
2813/2813 [==============================] - 1272s 452ms/step - loss: 0.3025 - accuracy: 0.8792 - val_loss: 0.3842 - val_accuracy: 0.8476


In [ ]:
y_pred = model.predict([X_test_input_ids, X_test_attn_masks])
y_pred_classes = np.argmax(y_pred, axis=1)


235/235 [==============================] - 69s 287ms/step


In [ ]:
y_pred_classes

array([2, 0, 2, ..., 0, 2, 0])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
categories = ['LQ_CLOSE', 'LQ_EDIT', 'HQ']

In [ ]:
print(classification_report(y_test, y_pred_classes, target_names=categories))

              precision    recall  f1-score   support

    LQ_CLOSE       0.76      0.84      0.80      2511
     LQ_EDIT       0.88      0.88      0.88      2483
          HQ       0.91      0.82      0.86      2506

    accuracy                           0.84      7500
   macro avg       0.85      0.84      0.85      7500
weighted avg       0.85      0.84      0.85      7500



In [ ]:
model.save('/content/drive/MyDrive/model/_PSO_model.h5') 

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/model/PSO_model_new.h5', custom_objects={"TFDistilBertModel": transformers.TFDistilBertModel})

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['LQ_CLOSE', 'LQ_EDIT', 'HQ']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
a = '''Why does HTML think “chucknorris” is a color? 

Why do certain random strings produce colors when entered as background colors in HTML?

For example, bgcolor="chucknorris" produces a red background:

<body bgcolor="chucknorris"> test </body>
Expand snippet
Conversely, bgcolor="chucknorr" produces a yellow background:

<body bgcolor="chucknorr"> test </body>
Expand snippet
This holds true across various browsers and platforms. What’s going on here?'''

In [ ]:
a = remove_special_chars(a)
a = remove_digits(a)
a = remove_underscore(a)
a = filter_words(a)

In [ ]:
processed_data = prepare_data(a, tokenizer)
result = make_prediction(model, processed_data=processed_data)
print(f"Predicted result: {result}")

1/1 [==============================] - 0s 57ms/step
Predicted result: LQ_EDIT
